In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [12]:
def load_to_csv(df: pd.DataFrame):
    df.to_csv('dataset.csv', index=False)

In [13]:
def ranges_splitter(ranges):
    ranges = ranges.replace(" ","")
    min, max = ranges.split('..')
    return min.replace(",",""), max.replace(",","")

In [14]:
def get_data_from_steam(app_id: str):
    pass

In [15]:
def get_data_from_steamspy_game_page(app_id: str):
    url = f"https://steamspy.com/year/{app_id}"
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    print('hello')
    
    pass

In [16]:
def get_data_from_steamspy_into_dataframe(year: int):
    url = f"https://steamspy.com/year/{year}"
    game_names = list()
    developer = list()
    minimum_owners = list()
    maximum_owners = list()
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    tbl = soup("table", attrs={"class":"table"})[0]
    for row in tbl("tr"):
        cells = row("td")
        if len(cells) > 0:
            game_names.append(cells[1].a.text)
            developer.append(cells[7].get_text())
            min, max = ranges_splitter(cells[5].get_text())
            minimum_owners.append(int(min))
            maximum_owners.append(int(max))    
            #get_data_from_steamspy_game_page(cells[1].a['href'])
    #print(len(game_names))
    df = pd.DataFrame({ 'Game Name': game_names, 
                        'Developer': developer,
                        'Minimum Owners': minimum_owners, 
                        'Maximum Owners': maximum_owners})
    return df

In [17]:
df = get_data_from_steamspy_into_dataframe(2021)
load_to_csv(df)

In [18]:
df = pd.read_csv('./dataset.csv')

In [19]:
df

,Game Name,Developer,Minimum Owners,Maximum Owners
0,OPUS: Echo of Starsong,SIGONO INC.,50000,100000
1,Chicory: A Colorful Tale,"Greg Lobanov, Alexis Dean-Jones, Lena Raine, M...",0,20000
2,Psychonauts 2,Double Fine Productions,100000,200000
3,It Takes Two,Hazelight,2000000,5000000
4,Lacuna – A Sci-Fi Noir Adventure,DigiTales Interactive,200000,500000
...,...,...,...,...
10389,Knights College,KAIJYU-09,20000,50000
10390,Flowers Blooming at the End of Summer,MidnightWorks,20000,50000
10391,Big Bia,Pixel House Games,20000,50000
10392,FINAL FANTASY III,Square Enix,50000,100000


In [20]:
df = get_data_from_steamspy_into_dataframe(2021)


KeyboardInterrupt: 